# Player Trueskill

### Helpful Resources
[Trueskill Documentation](https://trueskill.org/#version-0-4-4)  
[Trueskill for DOTA2 Players - Kaggle](https://www.kaggle.com/devinanzelmo/dota-2-skill-rating-with-trueskill)  
[Collection of Trueskill Examples](https://python.hotexamples.com/examples/trueskill/TrueSkill/-/python-trueskill-class-examples.html)  

In [1]:
# Housekeeping
import requests
import datetime
import pandas as pd
import numpy as np
import io
import os
import trueskill
import itertools
import math
import json
from sklearn.metrics import  f1_score, log_loss, precision_score, recall_score

import oeutils

pd.options.display.max_rows = 100
pd.options.display.max_columns = 500

workingdir = '../data/'
years = ['2018', '2019', '2020', '2021']

#oeutils.download(workingdir, years)
df = oeutils.read(workingdir, years)
df = oeutils.clean(df, split_on='player', keep_identities=False, keep_leagues=False, keep_columns=False)

/home/paulperrone/source/lol-analysis/notebooks/oeutils.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oe_data['gameid'] = oe_data['gameid'].str.strip()


In [2]:
df.shape

(188865, 117)

In [3]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
348,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,1,Blue,top,Duke,Invictus Gaming,Ornn,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,3,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9631,363.4340,0.150020,315.4340,NaN,13,0.4906,6,0.2264,2,NaN,NaN,10791,7272,274.4151,0.188550,8870.0,NaN,218.0,216.0,2,1,0,8.2264,3292.0,4758.0,90.0,3287.0,4640.0,88.0,5.0,118.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5280.0,7775.0,150.0,5060.0,7477.0,144.0,220.0,298.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
349,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,2,Blue,jng,Ning,Invictus Gaming,Kha'Zix,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,0,5,12,4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6924,261.2830,0.107854,693.8113,NaN,48,1.8113,24,0.9057,10,NaN,NaN,10831,7312,275.9245,0.189587,9775.0,NaN,194.0,36.0,158,96,8,7.3208,3170.0,3745.0,67.0,2988.0,3386.0,58.0,182.0,359.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4660.0,5588.0,100.0,4290.0,4957.0,80.0,370.0,631.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
350,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,3,Blue,mid,Rookie,Invictus Gaming,Orianna,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,8,12,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19985,754.1509,0.311303,274.6415,NaN,10,0.3774,7,0.2642,5,NaN,NaN,12165,8646,326.2642,0.224176,9310.0,NaN,266.0,246.0,20,16,4,10.0377,3633.0,5236.0,108.0,3378.0,5108.0,92.0,255.0,128.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,5389.0,7541.0,157.0,5211.0,8079.0,150.0,178.0,-538.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
351,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,4,Blue,bot,JackeyLove,Invictus Gaming,Ezreal,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,7,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23623,891.4340,0.367971,321.2075,NaN,14,0.5283,6,0.2264,2,NaN,NaN,14028,10509,396.5660,0.272482,13075.0,NaN,276.0,257.0,19,8,11,10.4151,3992.0,3286.0,97.0,3438.0,3018.0,88.0,554.0,268.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,6800.0,5569.0,159.0,5138.0,5263.0,134.0,1662.0,306.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
352,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,5,Blue,sup,Baolan,Invictus Gaming,Tahm Kench,A

In [4]:
# Consolidate league names
def standardize_leagues(league):
    if league == 'NA LCS':
        return 'LCS'
    elif league == 'EU LCS':
        return 'LEC'
    else:
        return league

df['league'] = df['league'].apply(lambda x: standardize_leagues(x))

In [5]:
df['patch_str'] = df['patch'].map(lambda x: str(x))

In [6]:
# LCS
#df = df[df['league'].isin(['LCS'])]

# LEC
#df = df[df['league'].isin(['LEC'])]

# LCK
#df = df[df['league'].isin(['LCK'])]

# LPL
#df = df[df['league'].isin(['LPL'])]

## Big Four
df = df[df['league'].isin(['LCK', 'LCS', 'LEC', 'LPL'])]


In [7]:
df.shape

(49680, 118)

In [8]:
df = df.sort_values(by=['date', 'player'])

In [9]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,patch_str
356,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,9,Red,bot,Able,Royal Never Give Up,Vayne,Zoe,Kalista,Miss Fortune,Lee Sin,Alistar,1590,0,2,3,2,4,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1509,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5597,211.2075,0.226985,457.0566,NaN,7,0.2642,6,0.2264,1,NaN,NaN,10050,6531,246.4528,0.280907,9675.0,NaN,248.0,227.0,21,20,0,9.3585,3438.0,3018.0,88.0,3992.0,3286.0,97.0,-554.0,-268.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,5138.0,5263.0,134.0,6800.0,5569.0,159.0,-1662.0,-306.0,-25.0,0.0,0.0,0.0,0.0,0.0,0.0,8.01
352,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,5,Blue,sup,Baolan,Invictus Gaming,Tahm Kench,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,0,1,9,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4035,152.2642,0.062852,375.6604,NaN,32,1.2075,8,0.3019,11,NaN,NaN,8348,4829,182.2264,0.125205,7025.0,NaN,84.0,84.0,0,0,0,3.1698,2211.0,2815.0,26.0,2252.0,2930.0,24.0,-41.0,-115.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3449.0,4046.0,45.0,3220.0,3574.0,30.0,229.0,472.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,8.01
348,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,1,Blue,top,Duke,Invictus Gaming,Ornn,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,3,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9631,363.4340,0.150020,315.4340,NaN,13,0.4906,6,0.2264,2,NaN,NaN,10791,7272,274.4151,0.188550,8870.0,NaN,218.0,216.0,2,1,0,8.2264,3292.0,4758.0,90.0,3287.0,4640.0,88.0,5.0,118.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5280.0,7775.0,150.0,5060.0,7477.0,144.0,220.0,298.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,8.01
351,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,4,Blue,bot,JackeyLove,Invictus Gaming,Ezreal,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,7,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23623,891.4340,0.367971,321.2075,NaN,14,0.5283,6,0.2264,2,NaN,NaN,14028,10509,396.5660,0.272482,13075.0,NaN,276.0,257.0,19,8,11,10.4151,3992.0,3286.0,97.0,3438.0,3018.0,88.0,554.0,268.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,6800.0,5569.0,159.0,5138.0,5263.0,134.0,1662.0,306.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,8.01
354,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,7,Red,jng,Kar

In [10]:
input_data = df[['gameid', 'date', 'year', 'patch', 'patch_str', 'league', 'side', 'team', 'player', 'position', 'champion', 'result']]
input_data.head()

,gameid,date,year,patch,patch_str,league,side,team,player,position,champion,result
356,2899-3157,2018-01-15 09:40:00,2018,8.01,8.01,LPL,Red,Royal Never Give Up,Able,bot,Vayne,0
352,2899-3157,2018-01-15 09:40:00,2018,8.01,8.01,LPL,Blue,Invictus Gaming,Baolan,sup,Tahm Kench,1
348,2899-3157,2018-01-15 09:40:00,2018,8.01,8.01,LPL,Blue,Invictus Gaming,Duke,top,Ornn,1
351,2899-3157,2018-01-15 09:40:00,2018,8.01,8.01,LPL,Blue,Invictus Gaming,JackeyLove,bot,Ezreal,1
354,2899-3157,2018-01-15 09:40:00,2018,8.01,8.01,LPL,Red,Royal Never Give Up,Karsa,jng,Jarvan IV,0


In [11]:
input_data.columns[input_data.isna().any()].tolist()

[]

In [12]:
columns = ['gameid', 'date', 'year', 'patch', 'patch_str', 'league', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name', \
           'blue_top_champ', \
           'blue_jng_champ', \
           'blue_mid_champ', \
           'blue_bot_champ', \
           'blue_sup_champ',\
           'red_top_champ', \
           'red_jng_champ', \
           'red_mid_champ', \
           'red_bot_champ', \
           'red_sup_champ']

player_rating = pd.DataFrame(columns=columns)

## Trueskill

In [13]:
ts = trueskill.TrueSkill(draw_probability=0.0)

In [14]:
player_ratings_dict = dict()
for i in input_data['player'].unique():
    player_ratings_dict[i] = ts.create_rating()

In [15]:
### Lambda Apply w Position as Key
def setup_match(df, gameid, position, champion):
    game_frame = df[df.gameid == gameid]
    blue_data = game_frame[game_frame.side == 'Blue']
    red_data = game_frame[game_frame.side == 'Red']
    #blue players
    blue_top = blue_data[blue_data.position == 'top']
    blue_top_name = blue_top.player.iloc[0]
    blue_top_champ = blue_top.champion.iloc[0]
    blue_jng = blue_data[blue_data.position == 'jng']
    blue_jng_name = blue_jng.player.iloc[0]
    blue_jng_champ = blue_jng.champion.iloc[0]
    blue_mid = blue_data[blue_data.position == 'mid']
    blue_mid_name = blue_mid.player.iloc[0]
    blue_mid_champ = blue_mid.champion.iloc[0]
    blue_bot = blue_data[blue_data.position == 'bot']
    blue_bot_name = blue_bot.player.iloc[0]
    blue_bot_champ = blue_bot.champion.iloc[0]
    blue_sup = blue_data[blue_data.position == 'sup']
    blue_sup_name = blue_sup.player.iloc[0]
    blue_sup_champ = blue_sup.champion.iloc[0]
    # red players
    red_top = red_data[red_data.position == 'top']
    red_top_name = red_top.player.iloc[0]
    red_top_champ = red_top.champion.iloc[0]
    red_jng = red_data[red_data.position == 'jng']
    red_jng_name = red_jng.player.iloc[0]
    red_jng_champ = red_jng.champion.iloc[0]
    red_mid = red_data[red_data.position == 'mid']
    red_mid_name = red_mid.player.iloc[0]
    red_mid_champ = red_mid.champion.iloc[0]
    red_bot = red_data[red_data.position == 'bot']
    red_bot_name = red_bot.player.iloc[0]
    red_bot_champ = red_bot.champion.iloc[0]
    red_sup = red_data[red_data.position == 'sup']
    red_sup_name = red_sup.player.iloc[0]
    red_sup_champ = red_sup.champion.iloc[0]
    # date, teams, result
    date = blue_data['date'].iloc[0]
    year = blue_data['year'].iloc[0]
    patch = blue_data['patch'].iloc[0]
    patch_str = blue_data['patch_str'].iloc[0]
    league = blue_data['league'].iloc[0]
    blue_team_result = blue_data['result'].iloc[0]
    blue_team = blue_data['team'].iloc[0]
    red_team = red_data['team'].iloc[0]
    # create pandas series for df
    match = pd.Series([gameid, date, year, patch, patch_str, league, \
                       blue_team, \
                       blue_top_name, \
                       blue_jng_name, \
                       blue_mid_name, \
                       blue_bot_name, \
                       blue_sup_name,\
                       blue_team_result, \
                       red_team, \
                       red_top_name, \
                       red_jng_name, \
                       red_mid_name, \
                       red_bot_name, \
                       red_sup_name, \
                       blue_top_champ, \
                       blue_jng_champ, \
                       blue_mid_champ, \
                       blue_bot_champ, \
                       blue_sup_champ,\
                       red_top_champ, \
                       red_jng_champ, \
                       red_mid_champ, \
                       red_bot_champ, \
                       red_sup_champ])
    return match


player_rating[['gameid', 'date', 'year', 'patch', 'patch_str', 'league', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name', \
           'blue_top_champ', \
           'blue_jng_champ', \
           'blue_mid_champ', \
           'blue_bot_champ', \
           'blue_sup_champ',\
           'red_top_champ', \
           'red_jng_champ', \
           'red_mid_champ', \
           'red_bot_champ', \
           'red_sup_champ']] = input_data.apply(lambda row: setup_match(input_data, row['gameid'], row['position'], row['champion']),axis=1)


In [16]:
player_rating.shape


(49680, 29)

In [17]:
player_rating = player_rating.reset_index(drop=True)
player_rating = player_rating.drop_duplicates(subset=['gameid'])
player_rating = player_rating.sort_values(by=['date'])

In [18]:
player_rating.head(20)

,gameid,date,year,patch,patch_str,league,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name,blue_top_champ,blue_jng_champ,blue_mid_champ,blue_bot_champ,blue_sup_champ,red_top_champ,red_jng_champ,red_mid_champ,red_bot_champ,red_sup_champ
0,2899-3157,2018-01-15 09:40:00,2018,8.01,8.01,LPL,Invictus Gaming,Duke,Ning,Rookie,JackeyLove,Baolan,1,Royal Never Give Up,Letme,Karsa,Xiaohu,Able,Ming,Ornn,Kha'Zix,Orianna,Ezreal,Tahm Kench,Maokai,Jarvan IV,Ryze,Vayne,Braum
10,2899-3158,2018-01-15 10:30:00,2018,8.01,8.01,LPL,Royal Never Give Up,Letme,Mlxg,Xiaohu,y4,Ming,1,Invictus Gaming,Duke,Ning,Rookie,JackeyLove,Baolan,Vladimir,Jarvan IV,Ryze,Tristana,Alistar,Maokai,Ivern,Azir,Kalista,Braum
20,2899-3159,2018-01-15 11:18:00,2018,8.01,8.01,LPL,Invictus Gaming,Duke,Ning,Rookie,JackeyLove,Baolan,0,Royal Never Give Up,Letme,Mlxg,Xiaohu,y4,Ming,Ornn,Kha'Zix,Taliyah,Ezreal,Leona,Shen,Rek'Sai,Ryze,Kalista,Alistar
30,2900-3160,2018-01-15 12:20:00,2018,8.01,8.01,LPL,Topsports Gaming,Lies,Karin,Ggoong,QiuQiu,Cat,0,Suning,LANGX,Tian,knight,Fury,Yoon,Gnar,Rengar,Ryze,Ezreal,Alistar,Vladimir,Kha'Zix,Zoe,Kalista,Tahm Kench
40,2900-3161,2018-01-15 13:17:00,2018,8.01,8.01,LPL,Topsports Gaming,Lies,Karin,Ggoong,QiuQiu,Cat,0,Suning,LANGX,Tian,knight,Fury,Yoon,Gnar,Zac,Ryze,Kog'Maw,Alistar,Vladimir,Kha'Zix,Zoe,Kalista,Taric
50,ESPORTSTMNT06/550246,2018-01-16 08:09:05,2018,8.01,8.01,LCK,Kingzone DragonX,Rascal,Peanut,Bdd,PraY,GorillA,0,KSV eSports,CuVee,Ambition,Crown,Ruler,CoreJJ,Ornn,Nidalee,Cassiopeia,Xayah,Tahm Kench,Gnar,Jarvan IV,Ryze,Varus,Braum
60,ESPORTSTMNT06/550247,2018-01-16 08:51:22,2018,8.01,8.01,LCK,KSV eSports,CuVee,Ambition,Crown,Ruler,CoreJJ,1,Kingzone DragonX,Rascal,Peanut,Bdd,PraY,GorillA,Gangplank,Rengar,Malzahar,Ezreal,Braum,Vladimir,Lee Sin,Ryze,Sivir,Taric
70,2901-3162,2018-01-16 09:18:00,2018,8.01,8.01,LPL,Bilibili Gaming,AmazingJ,Wisdomz,Athena,Jinjiao,Road,1,Vici Gaming,New,Crash,Easyhoon,Martin,Duan,Camille,Sejuani,Azir,Tristana,Braum,Fiora,Jarvan IV,Zoe,Kalista,Alistar
80,ESPORTSTMNT06/550268,2018-01-16 10:46:35,2018,8.01,8.01,LCK,Kongdoo Monster,Roach,Raise,Edge,SS,Secret,1,MVP,ADD,Beyond,Ian,MaHa,Max,Ornn,Jarvan IV,Zoe,Kog'Maw,Tahm Kench,Illaoi,Nidalee,Ryze,Twitch,Janna
90,ESPORTSTMNT06/550275,2018-01-16 11:39:09,2018,8.01,8.01,LCK,MVP,ADD,Beyond,Ian,MaHa,Max,0,Kongdoo Monster,Roach,Raise,Edge,SS,Secret,Gangplank,Kha'Zix,Ryze,Kog'Maw,Thresh,Gnar,Zac,Azir,Tristana,Shen


In [19]:
analyzed_gameids = {}

def win_probability(team1, team2, trueskill_global_env):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (trueskill_global_env.beta ** 2 ) + sum_sigma)
    return trueskill_global_env.cdf(delta_mu / denom)

def update_trueskill(rating_dict, gameid_dict, gameid, blue_team_result, \
                        blue_top_name, \
                        blue_jng_name, \
                        blue_mid_name, \
                        blue_bot_name, \
                        blue_sup_name,\
                        red_top_name, \
                        red_jng_name, \
                        red_mid_name, \
                        red_bot_name, \
                        red_sup_name):
    rating_groups = [(rating_dict[blue_top_name], \
                      rating_dict[blue_jng_name], \
                      rating_dict[blue_mid_name], \
                      rating_dict[blue_bot_name], \
                      rating_dict[blue_sup_name]), \
                     (rating_dict[red_top_name], \
                      rating_dict[red_jng_name], \
                      rating_dict[red_mid_name], \
                      rating_dict[red_bot_name], \
                      rating_dict[red_sup_name])]
    blue_team_win_prob = win_probability(rating_groups[0], rating_groups[1], ts)
    # Get Mu by position
    blue_top_mu = rating_dict[blue_top_name].mu
    blue_jng_mu = rating_dict[blue_jng_name].mu
    blue_mid_mu = rating_dict[blue_mid_name].mu
    blue_bot_mu = rating_dict[blue_bot_name].mu
    blue_sup_mu = rating_dict[blue_sup_name].mu
    red_top_mu = rating_dict[red_top_name].mu
    red_jng_mu = rating_dict[red_jng_name].mu
    red_mid_mu = rating_dict[red_mid_name].mu
    red_bot_mu = rating_dict[red_bot_name].mu
    red_sup_mu = rating_dict[red_sup_name].mu
    # Get Sigma by position
    blue_top_sigma = rating_dict[blue_top_name].sigma
    blue_jng_sigma = rating_dict[blue_jng_name].sigma
    blue_mid_sigma = rating_dict[blue_mid_name].sigma
    blue_bot_sigma = rating_dict[blue_bot_name].sigma
    blue_sup_sigma = rating_dict[blue_sup_name].sigma
    red_top_sigma = rating_dict[red_top_name].sigma
    red_jng_sigma = rating_dict[red_jng_name].sigma
    red_mid_sigma = rating_dict[red_mid_name].sigma
    red_bot_sigma = rating_dict[red_bot_name].sigma
    red_sup_sigma = rating_dict[red_sup_name].sigma
    # Update ratings
    if blue_team_result == 1:
        rated_rating_groups = ts.rate(rating_groups, ranks=[0, 1]) # for ranks 0 is winner 
    else:
        rated_rating_groups = ts.rate(rating_groups, ranks=[1, 0])
    # Get Updated Mu
    new_blue_top_mu = rated_rating_groups[0][0].mu
    new_blue_jng_mu = rated_rating_groups[0][1].mu
    new_blue_mid_mu = rated_rating_groups[0][2].mu
    new_blue_bot_mu = rated_rating_groups[0][3].mu
    new_blue_sup_mu = rated_rating_groups[0][4].mu
    new_red_top_mu = rated_rating_groups[1][0].mu
    new_red_jng_mu = rated_rating_groups[1][1].mu
    new_red_mid_mu = rated_rating_groups[1][2].mu
    new_red_bot_mu = rated_rating_groups[1][3].mu
    new_red_sup_mu = rated_rating_groups[1][4].mu
    # Get Updated Sigma
    new_blue_top_sigma = rated_rating_groups[0][0].sigma
    new_blue_jng_sigma = rated_rating_groups[0][1].sigma
    new_blue_mid_sigma = rated_rating_groups[0][2].sigma
    new_blue_bot_sigma = rated_rating_groups[0][3].sigma
    new_blue_sup_sigma = rated_rating_groups[0][4].sigma
    new_red_top_sigma = rated_rating_groups[1][0].sigma
    new_red_jng_sigma = rated_rating_groups[1][1].sigma
    new_red_mid_sigma = rated_rating_groups[1][2].sigma
    new_red_bot_sigma = rated_rating_groups[1][3].sigma
    new_red_sup_sigma = rated_rating_groups[1][4].sigma
    # Return values for new columns
    ts_update = pd.Series([blue_team_win_prob, \
           blue_top_mu, blue_top_sigma, \
           blue_jng_mu, blue_jng_sigma, \
           blue_mid_mu, blue_mid_sigma, \
           blue_bot_mu, blue_bot_sigma, \
           blue_sup_mu, blue_sup_sigma, \
           new_blue_top_mu, new_blue_top_sigma, \
           new_blue_jng_mu, new_blue_jng_sigma, \
           new_blue_mid_mu, new_blue_mid_sigma, \
           new_blue_bot_mu, new_blue_bot_sigma, \
           new_blue_sup_mu, new_blue_sup_sigma, \
           red_top_mu, red_top_sigma, \
           red_jng_mu, red_jng_sigma, \
           red_mid_mu, red_mid_sigma, \
           red_bot_mu, red_bot_sigma, \
           red_sup_mu, red_sup_sigma, \
           new_red_top_mu, new_red_top_sigma, \
           new_red_jng_mu, new_red_jng_sigma, \
           new_red_mid_mu, new_red_mid_sigma, \
           new_red_bot_mu, new_red_bot_sigma, \
           new_red_sup_mu, new_red_sup_sigma])
    # update the rating dictionary
    rating_dict[blue_top_name] = rated_rating_groups[0][0]
    rating_dict[blue_jng_name] = rated_rating_groups[0][1]
    rating_dict[blue_mid_name] = rated_rating_groups[0][2]
    rating_dict[blue_bot_name] = rated_rating_groups[0][3]
    rating_dict[blue_sup_name] = rated_rating_groups[0][4]
    rating_dict[red_top_name] = rated_rating_groups[1][0]
    rating_dict[red_jng_name] = rated_rating_groups[1][1]
    rating_dict[red_mid_name] = rated_rating_groups[1][2]
    rating_dict[red_bot_name] = rated_rating_groups[1][3]
    rating_dict[red_sup_name] = rated_rating_groups[1][4]
    # determine if use old data because of repeat
    if gameid in gameid_dict:
        return gameid_dict[gameid]
    else:
        gameid_dict[gameid] = ts_update
        return ts_update

player_rating[['blue_team_win_prob',\
           'blue_top_mu', 'blue_top_sigma', \
           'blue_jng_mu', 'blue_jng_sigma', \
           'blue_mid_mu', 'blue_mid_sigma', \
           'blue_bot_mu', 'blue_bot_sigma', \
           'blue_sup_mu', 'blue_sup_sigma', \
           'new_blue_top_mu', 'new_blue_top_sigma', \
           'new_blue_jng_mu', 'new_blue_jng_sigma', \
           'new_blue_mid_mu', 'new_blue_mid_sigma', \
           'new_blue_bot_mu', 'new_blue_bot_sigma', \
           'new_blue_sup_mu', 'new_blue_sup_sigma', \
           'red_top_mu', 'red_top_sigma', \
           'red_jng_mu', 'red_jng_sigma', \
           'red_mid_mu', 'red_mid_sigma', \
           'red_bot_mu', 'red_bot_sigma', \
           'red_sup_mu', 'red_sup_sigma', \
            'new_red_top_mu', 'new_red_top_sigma', \
           'new_red_jng_mu', 'new_red_jng_sigma', \
           'new_red_mid_mu', 'new_red_mid_sigma', \
           'new_red_bot_mu', 'new_red_bot_sigma', \
           'new_red_sup_mu', 'new_red_sup_sigma']] = player_rating.apply(lambda row: update_trueskill(player_ratings_dict, analyzed_gameids, row['gameid'], row['blue_team_result'], \
                        row['blue_top_name'], \
                        row['blue_jng_name'], \
                        row['blue_mid_name'], \
                        row['blue_bot_name'], \
                        row['blue_sup_name'],\
                        row['red_top_name'], \
                        row['red_jng_name'], \
                        row['red_mid_name'], \
                        row['red_bot_name'], \
                        row['red_sup_name']), axis=1)

In [20]:
def predict_blue_win(win_prob):
    if win_prob > .5:
        return 1
    else:
        return 0

player_rating['blue_predicted_to_win'] = player_rating['blue_team_win_prob'].apply(lambda x: predict_blue_win(x))
    
player_rating.shape

(4968, 71)

In [21]:
player_rating = player_rating.reset_index(drop=True)
player_rating.head(20)

,gameid,date,year,patch,patch_str,league,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name,blue_top_champ,blue_jng_champ,blue_mid_champ,blue_bot_champ,blue_sup_champ,red_top_champ,red_jng_champ,red_mid_champ,red_bot_champ,red_sup_champ,blue_team_win_prob,blue_top_mu,blue_top_sigma,blue_jng_mu,blue_jng_sigma,blue_mid_mu,blue_mid_sigma,blue_bot_mu,blue_bot_sigma,blue_sup_mu,blue_sup_sigma,new_blue_top_mu,new_blue_top_sigma,new_blue_jng_mu,new_blue_jng_sigma,new_blue_mid_mu,new_blue_mid_sigma,new_blue_bot_mu,new_blue_bot_sigma,new_blue_sup_mu,new_blue_sup_sigma,red_top_mu,red_top_sigma,red_jng_mu,red_jng_sigma,red_mid_mu,red_mid_sigma,red_bot_mu,red_bot_sigma,red_sup_mu,red_sup_sigma,new_red_top_mu,new_red_top_sigma,new_red_jng_mu,new_red_jng_sigma,new_red_mid_mu,new_red_mid_sigma,new_red_bot_mu,new_red_bot_sigma,new_red_sup_mu,new_red_sup_sigma,blue_predicted_to_win
0,2899-3157,2018-01-15 09:40:00,2018,8.01,8.01,LPL,Invictus Gaming,Duke,Ning,Rookie,JackeyLove,Baolan,1,Royal Never Give Up,Letme,Karsa,Xiaohu,Able,Ming,Ornn,Kha'Zix,Orianna,Ezreal,Tahm Kench,Maokai,Jarvan IV,Ryze,Vayne,Braum,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,1
1,2899-3158,2018-01-15 10:30:00,2018,8.01,8.01,LPL,Royal Never Give Up,Letme,Mlxg,Xiaohu,y4,Ming,1,Invictus Gaming,Duke,Ning,Rookie,JackeyLove,Baolan,Vladimir,Jarvan IV,Ryze,Tristana,Alistar,Maokai,Ivern,Azir,Kalista,Braum,0.301813,23.119255,8.118755,25.000000,8.333333,23.119255,8.118755,25.000000,8.333333,23.119255,8.118755,25.746780,7.881636,27.768235,8.076660,25.746780,7.881636,27.768235,8.076660,25.746780,7.881636,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,24.253220,7.881636,24.253220,7.881636,24.253220,7.881636,24.253220,7.881636,24.253220,7.881636,0
2,2899-3159,2018-01-15 11:18:00,2018,8.01,8.01,LPL,Invictus Gaming,Duke,Ning,Rookie,JackeyLove,Baolan,0,Royal Never Give Up,Letme,Mlxg,Xiaohu,y4,Ming,Ornn,Kha'Zix,Taliyah,Ezreal,Leona,Shen,Rek'Sai,Ryze,Kalista,Alistar,0.342114,24.253220,7.881636,24.253220,7.881636,24.253220,7.881636,24.253220,7.881636,24.253220,7.881636,23.027800,7.715670,23.027800,7.715670,23.027800,7.715670,23.027800,7.715670,23.027800,7.715670,25.746780,7.881636,27.768235,8.076660,25.746780,7.881636,27.768235,8.076660,25.746780,7.881636,26.972200,7.715670,29.055043,7.897928,26.972200,7.715670,29.055043,7.897928,26.972200,7.715670,0
3,2900-3160,2018-01-15 12:20:00,2018,8.01,8.01,LPL,Topsports Gaming,Lies,Karin,Ggoong,QiuQiu,Cat,0,Suning,LANGX,Tian,knight,Fury,Yoon,Gnar,Rengar,Ryze,Ezreal,Alistar,Vladimir,Kha'Zix,Zoe,Kalista,Tahm Kench,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,1
4,2900-3161,2018-01-15 13:17:00,2018,8.01,8.01,LPL,Topsports Gaming,Lies,Karin,Ggoong,QiuQiu,Cat,0,Suning,LANGX,Tian,knight,Fury,Yoon,Gnar,Zac,Ryze,Kog'Maw,Alistar,Vladimir,Kha'Zix,Zoe,Kalista,Taric,0.257286,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,23.119255,8.118755,22.127021,7.966146,22.127021,7.966146,22.127021,7.966146,22.127021,7.966146,22.127021,7.966146,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,26.880745,8.118755,27.872979,7.966146,27.872979,7.966146,27.872979,7.966146,27.872979,7.966

### Check Accuracy

In [22]:
# All Data
y_true = player_rating['blue_team_result']
y_pred_bool = player_rating['blue_predicted_to_win']
y_pred_pct = player_rating['blue_team_win_prob']

print('---Metrics for all data---')
print('F1 Score: ', f1_score(y_true, y_pred_bool))
print('Precision Score: ', precision_score(y_true, y_pred_bool))
print('Recall Score: ', recall_score(y_true, y_pred_bool))
print('-')
print('Log Loss: ', log_loss(y_true, y_pred_pct))
print()
# Primed on 2018-2019, Results Measured on 2020-2021
player_rating_2020on = player_rating[player_rating['year'] >= 2020]
y_true = player_rating_2020on['blue_team_result']
y_pred_bool = player_rating_2020on['blue_predicted_to_win']
y_pred_pct = player_rating_2020on['blue_team_win_prob']

print('---Metrics for 2020-2021 data---')
print('F1 Score: ', f1_score(y_true, y_pred_bool))
print('Precision Score: ', precision_score(y_true, y_pred_bool))
print('Recall Score: ', recall_score(y_true, y_pred_bool))
print('-')
print('Log Loss: ', log_loss(y_true, y_pred_pct))


---Metrics for all data---
F1 Score:  0.6283168700368004
Precision Score:  0.6480223731522173
Recall Score:  0.6097744360902255
-
Log Loss:  0.6931538058179382

---Metrics for 2020-2021 data---
F1 Score:  0.6204081632653061
Precision Score:  0.6516613076098606
Recall Score:  0.5920155793573515
-
Log Loss:  0.6801928692183896


### Calculate Future Winner from Input Teams

Team1 and Team2 input tuples for win_probability function  

(trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333))  

(trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333))

#print(player_ratings_dict)

TSM = ['Huni', 'Spica', 'PowerOfEvil', 'Lost', 'SwordArt']
FLY = ['Licorice', 'Josedeodo', 'Palafox', 'Johnsun', 'Diamond']
TSM_RATINGS = []
FLY_RATINGS = []

for player in TSM:
    TSM_RATINGS.append(player_ratings_dict[player])

for player in FLY:
    FLY_RATINGS.append(player_ratings_dict[player])
    
TSM_TUPLE = tuple(TSM_RATINGS)
FLY_TUPLE = tuple(FLY_RATINGS)

team_1_win_prob = win_probability(TSM_TUPLE, FLY_TUPLE, ts)
print('TSM Win Probability :' + str(team_1_win_prob))

In [23]:
#print(analyzed_gameids)

In [24]:
#player_rating.to_csv('./big4_player_trueskill_2018-2021.csv', index=False)

## Average Rating Change for a Player on a given Champion by League by Patch

### Create Dataframe

In [25]:
leagues = input_data['league'].unique()
patches = input_data['patch_str'].unique()
champions = input_data['champion'].unique()

champ_dict = dict()
for i in champions:
    champ_dict[i] = []

patch_dict = dict()
for i in patches:
    patch_dict[i] = champ_dict

league_dict = dict()
for i in leagues:
    league_dict[i] = patch_dict

In [26]:
for league in leagues:
    for patch in patches:
        patch_df = player_rating.loc[(player_rating['league'] == league) & (player_rating['patch_str'] == patch)]
        champion_patch_dict = dict()
        for i in champions:
            champion_patch_dict[i] = []
        for index, row in patch_df.iterrows():
            champion_patch_dict[row['blue_top_champ']].append(row['new_blue_top_mu'] - row['blue_top_mu'])
            champion_patch_dict[row['blue_jng_champ']].append(row['new_blue_jng_mu'] - row['blue_jng_mu'])
            champion_patch_dict[row['blue_mid_champ']].append(row['new_blue_mid_mu'] - row['blue_mid_mu'])
            champion_patch_dict[row['blue_bot_champ']].append(row['new_blue_bot_mu'] - row['blue_bot_mu'])
            champion_patch_dict[row['blue_sup_champ']].append(row['new_blue_sup_mu'] - row['blue_sup_mu'])
            champion_patch_dict[row['red_top_champ']].append(row['new_red_top_mu'] - row['red_top_mu'])
            champion_patch_dict[row['red_jng_champ']].append(row['new_red_jng_mu'] - row['red_jng_mu'])
            champion_patch_dict[row['red_mid_champ']].append(row['new_red_mid_mu'] - row['red_mid_mu'])
            champion_patch_dict[row['red_bot_champ']].append(row['new_red_bot_mu'] - row['red_bot_mu'])
            champion_patch_dict[row['red_sup_champ']].append(row['new_red_sup_mu'] - row['red_sup_mu'])
        league_dict[league][patch] = champion_patch_dict

In [27]:
with open('./trueskill/rating_changes_by_champ/league_patch_champ_rating_deltas_.json', 'w') as json_file:
    json.dump(league_dict, json_file)

In [28]:
if league_dict == False:
    with open('./trueskill/rating_changes_by_champ/league_patch_champ_rating_deltas.json', 'r') as json_file:
        league_dict = json_file

In [29]:
columns = ['league', 'patch']
champions = input_data['champion'].unique()
for champ in champions:
    columns.append(champ)

In [30]:
champ_ratings_league_patch = pd.DataFrame(columns=columns)

In [31]:
for league in leagues:
    for patch in patches:
        row = pd.DataFrame(columns=columns)
        row.at[0, 'league'] = league
        row.at[0, 'patch'] = patch
        for champ in champions:
            if len(league_dict[league][patch][champ]) >= 2:
                row.at[0, champ] = (sum(league_dict[league][patch][champ]) / len(league_dict[league][patch][champ]))
            else:
                row.at[0, champ] = np.nan
        champ_ratings_league_patch = pd.concat([champ_ratings_league_patch, row], axis=0)


In [32]:
#champ_ratings_league_patch.to_csv('./trueskill/rating_changes_by_champ/rating_changes_by_champ.csv', index=False)

### Analyze Average Rating Change

In [34]:
champ_ratings_league_patch = pd.read_csv('./trueskill/rating_changes_by_champ/rating_changes_by_champ.csv')

In [35]:
champ_ratings_league_patch.shape

(164, 152)

In [36]:
champ_ratings_league_patch.head()

,league,patch,Vayne,Tahm Kench,Ornn,Ezreal,Jarvan IV,Maokai,Braum,Kha'Zix,Orianna,Ryze,Kalista,Vladimir,Alistar,Ivern,Azir,Tristana,Leona,Shen,Rek'Sai,Taliyah,Rengar,Gnar,Zoe,Zac,Kog'Maw,Taric,Cassiopeia,Nidalee,Xayah,Varus,Malzahar,Gangplank,Lee Sin,Sivir,Camille,Fiora,Sejuani,Illaoi,Twitch,Janna,Thresh,Rumble,Miss Fortune,Veigar,Karma,Jhin,Lucian,Caitlyn,Jax,Lulu,Shyvana,Corki,Jayce,Rakan,Kassadin,Urgot,Galio,Cho'Gath,LeBlanc,Evelynn,Zilean,Syndra,Aurelion Sol,Gragas,Morgana,Xerath,Graves,Kled,Sion,Bard,Kennen,Jinx,Skarner,Kayn,Riven,Poppy,Xin Zhao,Trundle,Nasus,Soraka,Nunu & Willump,Yorick,Blitzcrank,Brand,Twisted Fate,Ziggs,Anivia,Malphite,Yasuo,Draven,Viktor,Ekko,Vel'Koz,Aatrox,Ashe,Olaf,Volibear,Swain,Renekton,Darius,Kindred,Nocturne,Kai'Sa,Nami,Singed,Fizz,Talon,Pyke,Mordekaiser,Dr. Mundo,Nautilus,Irelia,Master Yi,Fiddlesticks,Karthus,Elise,Heimerdinger,Kayle,Hecarim,Akali,Lissandra,Lux,Udyr,Zyra,Zed,Wukong,Quinn,Teemo,Ahri,Amumu,Katarina,Neeko,Annie,Sylas,Pantheon,Vi,Sona,Yuumi,Qiyana,Garen,Diana,Senna,Aphelios,Sett,Tryndamere,Lillia,Samira,Yone,Rell,Seraphine
0,LPL,8.01,NaN,0.860565,0.047682,0.109402,-1.278189,1.134098,-0.107311,-0.097095,-1.068043,0.236870,0.485959,-1.388618,-1.338614,NaN,0.510166,0.746805,NaN,0.376144,NaN,-0.385483,-1.635114,-0.368591,-0.280004,0.911282,0.141181,0.677957,-0.159777,NaN,NaN,-1.411819,0.074065,0.796707,NaN,NaN,NaN,NaN,0.278236,NaN,NaN,NaN,-0.075809,NaN,NaN,NaN,NaN,NaN,NaN,-1.366227,NaN,NaN,NaN,NaN,-1.351493,NaN,NaN,NaN,NaN,-1.851666,NaN,1.615182,0.444255,0.706190,NaN,1.812947,-0.387792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LPL,8.02,NaN,-0.586777,0.236843,0.119736,-0.179772,NaN,0.877672,0.317244,0.318538,-0.832180,0.223655,-0.493224,-0.266310,NaN,-0.724794,-1.193128,NaN,1.160094,NaN,0.593897,NaN,0.191797,2.086645,0.390621,-0.026565,-0.880429,1.073863,NaN,-0.303118,0.556744,NaN,0.576071,NaN,NaN,-0.399136,NaN,-0.527765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.965437,NaN,NaN,0.929993,NaN,-0.303118,NaN,NaN,0.163307,NaN,NaN,NaN,NaN,NaN,NaN,-1.942594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.69577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LPL,8.03,NaN,0.692051,-0.142914,0.075194,NaN,NaN,0.637541,1.078195,0.785914,0.231188,-1.020812,-0.846549,-0.797396,NaN,0.833357,0.114101,NaN,NaN,NaN,0.129734,NaN,-0.007468,-0.233275,-0.991851,NaN,-1.441750,NaN,NaN,NaN,0.662428,NaN,2.863454,NaN,NaN,0.163191,-2.224807,-0.527391,NaN,NaN,0.412832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.385696,0.575214,NaN,NaN,-2.775377,NaN,0.378461,NaN,NaN,1.729022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.218921,NaN,NaN,NaN,-0.732770,NaN,NaN,NaN,1.203020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.509864,-0.36981,0.063597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LPL,8.04,NaN,-0.906892,NaN,-0.085709,NaN,-0.536900,-0.317212,-0.834002,0.500996,-0.200948,NaN,NaN,-1.460840,NaN,-0.544405,-1.669444,NaN,0.373048,NaN,2.092754,NaN,-0.236810,NaN,-0.060548,NaN,NaN,-0.353494,NaN,0.969728,-0.629512,NaN,1.094035,-1.231498,NaN,-1.035546,NaN,0.729079,NaN,NaN,NaN,0.677072,NaN,NaN,NaN,-0.096279,NaN,NaN,0.306158,NaN,NaN,NaN,-1.772786,NaN,0.628570,NaN,NaN,-0.069243,0.467714,NaN,NaN,NaN,0.222062,NaN,NaN,0.536783,NaN,NaN,NaN,-0.337478,NaN,NaN,NaN,0.183391,NaN,NaN,NaN,NaN,-2.135293,NaN,NaN,NaN,NaN,